In [1]:
!set HTTP_PROXY=http://127.0.0.1: 7890
!set HTTPS_PROXY=https://127.0.0.1: 7890

In [2]:
import praw
import re

In [3]:
reddit = praw.Reddit(
    client_id="oJd0bJxDJiyBpsGOi7u0rg",
    client_secret="sPJ3VJqMavUNUfXbB7b0ipU34Z-H1g",
    user_agent="testscript by u/fakebot3",
)

In [4]:
def get_hot_posts(subreddit_name="all", limit=10):
    """
    获取指定子版块的热门帖子\n
    :param subreddit_name: 子版块名称（默认 'all' 即全站）\n
    :param limit: 获取的帖子数量\n
    :return: 帖子信息列表
    """
    subreddit = reddit.subreddit(subreddit_name)
    hot_posts = []
    notFoundList = []

    print(f"🔥 正在获取 r/{subreddit_name} 的热门帖子...\n")

    try:
        for post in subreddit.hot(limit=limit):
        # 提取关键信息
            post_info = {
                "标题": post.title,
                "作者": f"u/{post.author.name}" if post.author else "[已删除]",
                "分数": post.score,
                "评论数": post.num_comments,
                "链接": f"https://reddit.com{post.permalink}",
                "NSFW": post.over_18,
                "发布时间": post.created_utc,  # Unix 时间戳
                "媒体类型": "视频" if post.is_video else "图片" if post.url.endswith(('jpg', 'png', 'gif')) else "文本/链接"
            }
            hot_posts.append(post_info)
    except Exception as e:
        notFoundList.append(subreddit_name)
        return [], notFoundList

    return hot_posts, []

排除的话题：体育（视频太多了）
添加的话题：篮球，Trump

In [9]:
# 示例：获取 r/all 的 10 条热门帖子
posts, notFoundList = get_hot_posts(subreddit_name="basketball", limit=10)

# 打印结果
for idx, post in enumerate(posts, 1):
    print(f"【{idx}】{post['标题']}")
    print(f"    👍 分数: {post['分数']} | 💬 评论: {post['评论数']} | 👤 作者: {post['作者']}")
    print(f"    🔗 链接: {post['链接']}")
    print(f"    🏷️ NSFW: {'是' if post['NSFW'] else '否'} | 🎬 媒体类型: {post['媒体类型']}\n")

🔥 正在获取 r/basketball 的热门帖子...

【1】New Rule: Do Not ask for Medical Advice
    👍 分数: 82 | 💬 评论: 11 | 👤 作者: u/Commandant1
    🔗 链接: https://reddit.com/r/Basketball/comments/1jo7aau/new_rule_do_not_ask_for_medical_advice/
    🏷️ NSFW: 否 | 🎬 媒体类型: 文本/链接

【2】Official Shoes, Gear, Equipment, etc... Megathread
    👍 分数: 2 | 💬 评论: 3 | 👤 作者: u/Commandant1
    🔗 链接: https://reddit.com/r/Basketball/comments/1k1izx2/official_shoes_gear_equipment_etc_megathread/
    🏷️ NSFW: 否 | 🎬 媒体类型: 文本/链接

【3】why just wnba balls smaller and not anything else?
    👍 分数: 63 | 💬 评论: 90 | 👤 作者: u/kslmbda
    🔗 链接: https://reddit.com/r/Basketball/comments/1k2fidh/why_just_wnba_balls_smaller_and_not_anything_else/
    🏷️ NSFW: 否 | 🎬 媒体类型: 文本/链接

【4】Adam Silver Says Mavs Aren't Leaving Dallas, No 'Ulterior Motives' for Luka Trade
    👍 分数: 80 | 💬 评论: 50 | 👤 作者: u/Edm_vanhalen1981
    🔗 链接: https://reddit.com/r/Basketball/comments/1k2b0ti/adam_silver_says_mavs_arent_leaving_dallas_no/
    🏷️ NSFW: 否 | 🎬 媒体类型: 文本/链接

【5】

In [10]:
posts_, _ = get_hot_posts(subreddit_name="Trump", limit=10)
posts += posts_

🔥 正在获取 r/Trump 的热门帖子...



In [23]:
for post in posts:
    print("(\"\"\"" + post['标题'] + "\"\"\"," + "\"\"\"" + post['链接'] + "\"\"\"),")

("""New Rule: Do Not ask for Medical Advice""","""https://reddit.com/r/Basketball/comments/1jo7aau/new_rule_do_not_ask_for_medical_advice/"""),
("""Official Shoes, Gear, Equipment, etc... Megathread""","""https://reddit.com/r/Basketball/comments/1k1izx2/official_shoes_gear_equipment_etc_megathread/"""),
("""why just wnba balls smaller and not anything else?""","""https://reddit.com/r/Basketball/comments/1k2fidh/why_just_wnba_balls_smaller_and_not_anything_else/"""),
("""Adam Silver Says Mavs Aren't Leaving Dallas, No 'Ulterior Motives' for Luka Trade""","""https://reddit.com/r/Basketball/comments/1k2b0ti/adam_silver_says_mavs_arent_leaving_dallas_no/"""),
("""Lost my College Intramural Championship for calling a timeout when we didn't have one :(""","""https://reddit.com/r/Basketball/comments/1k2qvcv/lost_my_college_intramural_championship_for/"""),
("""how should i start watching matches??(idk)""","""https://reddit.com/r/Basketball/comments/1k2ofyn/how_should_i_start_watching_matchesi

In [7]:
notFoundList

['e-sports']

In [ ]:
def extract_post_id(post_url):
    """
    从 Reddit 帖子链接中提取帖子 ID\n
    :param post_url: Reddit 帖子链接\n
    :return: 帖子 ID
    """
    match = re.search(r'comments/([a-zA-Z0-9]+)/', post_url)
    return match.group(1) if match else None

In [ ]:
# 通过 id 获得帖子的内容
def get_content(post_id):
    """
    获取指定帖子的内容\n
    :param post_id: 帖子 ID\n
    :return: 帖子内容
    """
    submission = reddit.submission(id=post_id)
    content = submission.selftext
    return content

In [ ]:
def get_posts_title_content(posts):
    """
    获取帖子内容\n
    :param posts: 帖子信息列表
    :return: 帖子 ID、标题和内容的元组列表
    """
    posts_id_title_contents = []
    for post in posts:
        submission = reddit.submission(extract_post_id(post['链接']))
        content = submission.selftext
        posts_id_title_contents.append((extract_post_id(post['链接']), post['标题'], content, post['链接']))
    return posts_id_title_contents

In [12]:

# 全站的热搜，对应页面的“今日热搜”
posts_all, notFoundList = get_hot_posts(subreddit_name="all", limit=10)

🔥 正在获取 r/all 的热门帖子...



In [16]:
for post in posts_all:
    print(post['标题'])

Japan Just Gave Ukraine $3 Billion from Russian assets
THE DALLAS MAVERICKS HAVE BEEN ELIMINATED FROM THE 2025 NBA CHAMPIONSHIP CONTENTION
Wholesome ex-couple
So….I found this in my garage this morning. Ummm
NASAs first six women astronauts. February 1979
Shine on.
Boston’s Old North Church on the 250th anniversary of Paul Revere’s midnight ride
Trump brags gas is down to $1.98 a gallon in a ‘couple of states.’ It isn’t — anywhere
On the Old North Church last night in Boston.
Kneecap open Coachella Gig with strong Messaging


In [21]:
for post in posts_all:
    print("('" + post['标题'] + "'," + "'" + post['链接'] + "'),")

('Japan Just Gave Ukraine $3 Billion from Russian assets','https://reddit.com/r/worldnews/comments/1k2olxh/japan_just_gave_ukraine_3_billion_from_russian/'),
('THE DALLAS MAVERICKS HAVE BEEN ELIMINATED FROM THE 2025 NBA CHAMPIONSHIP CONTENTION','https://reddit.com/r/nba/comments/1k2nr7a/the_dallas_mavericks_have_been_eliminated_from/'),
('Wholesome ex-couple','https://reddit.com/r/MadeMeSmile/comments/1k2m06j/wholesome_excouple/'),
('So….I found this in my garage this morning. Ummm','https://reddit.com/r/cats/comments/1k2l5mh/soi_found_this_in_my_garage_this_morning_ummm/'),
('NASAs first six women astronauts. February 1979','https://reddit.com/r/OldSchoolCool/comments/1k2lkbj/nasas_first_six_women_astronauts_february_1979/'),
('Shine on.','https://reddit.com/r/BlackPeopleTwitter/comments/1k2ln0e/shine_on/'),
('Boston’s Old North Church on the 250th anniversary of Paul Revere’s midnight ride','https://reddit.com/r/pics/comments/1k2ldt3/bostons_old_north_church_on_the_250th_anniversary/